# `sbi` -- a toolkit for simulation-based inference

Demo at https://github.com/mackelab/sbi

### Installation
`pip install sbi`

## Simple vs. flexible interface

### What we need to run `sbi`
- **prior**: `PyTorch` distribution or `scipy` distribution over parameters
- **simulator**: python function `f(parameters) -> data`

### Simple interface
`parameter_posterior = infer(simulator, prior, method='SNPE', num_simulations=100)`

## Flexible interface 

### Three steps
1. (pre-)simulate
2. train density estimator
3. obtain posterior

### 1. Simulate

In [ ]:
import torch
from torch import zeros, ones

import sbi
from sbi.utils import BoxUniform
from sbi.inference import prepare_for_sbi, simulate_for_sbi, SNPE
from sbi.analysis import pairplot

In [ ]:
# Uniform prior in three dimensions.
prior = BoxUniform(-ones(3), ones(3))

# Gaussian simulator.
def simulator(theta, scale=0.1):
    return scale * torch.randn(theta.shape) + theta

In [ ]:
# Run simulations ourselves
num_simulations = 100
# Check simulator and prior (batch dimensions, processing scipy prior etc.)
simulator, prior = prepare_for_sbi(simulator, prior)

theta = prior.sample((num_simulations, ))

x = simulator(theta)

In [ ]:
# or with sbi utils (parallelization!)
theta, x = simulate_for_sbi(simulator, prior, num_simulations, num_workers=1)

### 2. Train density estimator

In [ ]:
# Create inference object: choose method and estimator
inferer = SNPE(prior, density_estimator="mdn", device="cpu")  # SNLE, SNRE

In [ ]:
# Append training data
inferer = inferer.append_simulations(theta, x)

# Train
density_estimator = inferer.train()  # Lots of training settings.

### 3. Obtain posterior

In [ ]:
# Build posterior using trained density estimator
posterior = inferer.build_posterior(density_estimator)  # Posterior sampling settings.

In [ ]:
# Generate samples
theta_o = prior.sample((1,))
x_o = simulator(theta_o)
samples = posterior.sample((1000,), x=x_o)

In [ ]:
pairplot(samples, points=theta_o, points_colors=["k"], upper="scatter", limits=[[-1, 1]], figsize=(5,5));

## Practical: Inference with SNPE, SNLE and SNRE.

### inference problem
$$ \text{prior: } \theta \sim \mathcal{N}(0, I) \\ 
\text{simulator: } x \sim \mathcal{N}(\theta, \; 0.1 \; I)
$$

1. [optional] Explore the `sbi` documentation, have a look at the API references on the website etc. 

2. Repeat the steps above to run inference with `SNPE`.

3. Try the other two methods you learned about: [`SNLE`](https://www.mackelab.org/sbi/reference/#sbi.inference.snle.snle_a.SNLE_A), which estimates the likelihood to then run MCMC, and [`SNRE`](https://www.mackelab.org/sbi/reference/#sbi.inference.snre.snre_b.SNRE_B) which trains a classifier to approximate likelihood ratios to then run MCMC. 
4. Do you notice any differences when drawing posterior samples, if yes, why?

5. [optional] Work through a couple of the notebooks in the `sbi` repository (`/tutotials`).